In [1]:
# Import libraries

import numpy as np
import matplotlib.pyplot as plt

from data import pipeline
from data.window import SlidingWindow

# Load the TensorBoard notebook extension
%load_ext tensorboard

import tensorflow as tf
import datetime

# Clear any logs from previous runs
# !rm -rf ./logs/

# Data preparation

In [2]:
# Load / Build dataset

X_train, Y_train, X_test, Y_test = pipeline.load("LSTM") # load / buildAndSave / build
print(X_train.shape, Y_train.shape)

Loading LSTM dataset
Done!
(1050578, 4, 8) (1050578, 1)


In [3]:
# Normalisation

from sklearn.preprocessing import StandardScaler

X_train = X_train.reshape(-1, 32)
X_test = X_test.reshape(-1, 32)

sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

sc_y = StandardScaler()
Y_train = sc_y.fit_transform(Y_train.reshape(-1, 1))

X_train = X_train.reshape(-1, 4, 8)
X_test = X_test.reshape(-1, 4, 8)

print(X_train.shape, Y_train.shape)

(1050578, 4, 8) (1050578, 1)


# Training

In [4]:
# Import libraries for training and tuning

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from keras_tuner import BayesianOptimization, Objective

In [5]:
# Define how to implement the model

def implement_model(units, learning_rate):
    model = Sequential()
    model.add(LSTM(units= units,
                   activation= 'relu',
                   input_shape= (4, 8)))
    model.add(Dense(1))
    model.compile(optimizer= Adam(learning_rate= learning_rate),
                  loss= 'mean_squared_error',
                  metrics= [RootMeanSquaredError()])
    return model


# Define how to build the model with hyperparameters

def build_model(hp):
    # Tune the number of units in the LSTM per cell in the layer
    # units between 50-90
    hp_units = hp.Int('units', min_value= 50, max_value= 90, step= 2)

    # Tune the learning rate for the optimizer
    # learning_rate from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
    model = implement_model(hp_units, hp_learning_rate)
    return model


# Define the tuner

tuner = BayesianOptimization(
    hypermodel= build_model,
    objective= Objective('val_root_mean_squared_error', direction= 'min'),
    num_initial_points= 2,
    executions_per_trial= 3,
    project_name= 'lstm_automl',
    overwrite= False # True at first search
)

INFO:tensorflow:Reloading Oracle from existing project .\lstm_automl\oracle.json
INFO:tensorflow:Reloading Tuner from .\lstm_automl\tuner0.json


In [ ]:
# Reload the tuner

tuner.reload() # Deactivate at first search


# Search the best hyperparameters

history=tuner.search(x= X_train, y= Y_train, epochs= 10, validation_data=(X_test, Y_test))


# Get the results of the tuning

tuner.results_summary()
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""The best:
- dimension of the hidden state found is {best_hps.get('units')}
- learning rate found is {best_hps.get('learning_rate')}""")


Search: Running Trial #9

Value             |Best Value So Far |Hyperparameter
70                |68                |units
0.005834          |0.0064156         |learning_rate

Epoch 1/10
32831/32831 [==============================] - 107s 3ms/step - loss: 0.0812 - root_mean_squared_error: 0.2850 - val_loss: 2013.8969 - val_root_mean_squared_error: 44.8765
Epoch 2/10
32831/32831 [==============================] - 107s 3ms/step - loss: 0.0635 - root_mean_squared_error: 0.2519 - val_loss: 2008.9790 - val_root_mean_squared_error: 44.8216
Epoch 3/10
32831/32831 [==============================] - 110s 3ms/step - loss: 0.0591 - root_mean_squared_error: 0.2430 - val_loss: 2008.0253 - val_root_mean_squared_error: 44.8110
Epoch 4/10
32831/32831 [==============================] - 120s 4ms/step - loss: 0.0558 - root_mean_squared_error: 0.2363 - val_loss: 2009.4447 - val_root_mean_squared_error: 44.8268
Epoch 5/10
32831/32831 [==============================] - 118s 4ms/step - loss: 0.2397 - root_m

In [ ]:
# Reload the model from the tuner

tuner.reload()
model = tuner.hypermodel.build(best_hps)


# Implement the model with default hyperparameters

#model = implement_model(units= 70, learning_rate= 0.005)


# Initialise Tensorboard to visualise the loss

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
# Train the model

history = model.fit(X_train, Y_train, epochs= 10, callbacks= [tensorboard_callback])
%tensorboard --logdir logs/gradient_tape

# Evaluation

In [ ]:
# Reduce the size of the test dataset

X_test = X_test[:999]
Y_test = Y_test[:999]
print(X_test.shape, Y_test.shape)

In [ ]:
decades_per_year = 37
Y_naive = [Y_test[4:4+decades_per_year].mean()]*3
for decade in range(3, len(Y_test), decades_per_year):
    growth_avg = Y_test[decade:decade + decades_per_year].mean()
    Y_naive += [growth_avg] * decades_per_year

print('Test RMSE with naive model : %.3f' % sqrt(mean_squared_error(Y_test, Y_naive[3:])))

In [ ]:
sw = SlidingWindow(X_test, reset_cycle= 37)

pred = model.predict(sw.values.reshape(-1, 4, 8), verbose=0) # iterator: 0
Y_preds = [pred]

last_progression = 0
print(0, '%')
while sw.next(pred): # iterator: t-1
    arg = sw.values.reshape(-1, 4, 8)
    pred = model.predict(sw.values.reshape(-1, 4, 8), verbose=0) # iterator: t
    if np.isnan(pred):
        print(arg)
    Y_preds.append(pred)
        
    progression = (sw.it+1) / len(X_test) * 100
    if progression - last_progression > 1 and not int(progression) % 10:
        print(int(progression), '%')
        last_progression = progression

Y_preds = np.array(Y_preds).reshape(-1)

print('Test RMSE : %.3f' % sqrt(mean_squared_error(Y_test, Y_preds)))

In [ ]:
Y_preds = sc_y.inverse_transform(Y_preds.reshape(1, -1))
Y_preds = Y_preds.reshape(-1)
Y_test = Y_test.reshape(-1)

In [ ]:
year = 5
decades = [decade for decade in range(37)]
plt.plot(decades, [None]*3 + list(Y_test[37*year:37*(year+1)-3]), color="blue", label= "raw")
plt.plot(decades, [9.57]*3 + list(Y_preds[37*year:37*(year+1)-3]), color="red", label= "LSTM")
plt.plot(decades, Y_naive[37*year:37*(year+1)], color="green", label= "naive model")
plt.xlabel("Decades")
plt.ylabel("Growth")
plt.legend()
plt.show()